In [ ]:
!pip install ultralytics
!pip install pyngrok

In [ ]:
%cd /content
!git clone https://github.com/seraj94ai/Flask-streaming-Pedestrians-detection-using-python-opencv-.git

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
from ultralytics import YOLO
import torch
import cv2
import numpy as np
import pathlib
import matplotlib.pyplot as plt
#img = cv2.imread('/content/cam.jpg')
model = YOLO("yolov8n.pt")
names = model.names #classes
def detection_yolov8_frame(img):
  results = model(img)
  for result in results:
      boxes = result.boxes  # Boxes object for bbox outputs
      masks = result.masks  # Masks object for segmentation masks outputs
      probs = result.probs  # Class probabilities for classification outputs
      li = result.orig_img

  # Run the YOLOv8 model to detect objects in the image
  #results = model.predict(img)

  # Loop through each detected object
  x1, y1, x2, y2 = 0,0,0,0
  for i in range(len(result.boxes.xyxy)):
      # Get the bounding box coordinates
      if names[int(results[0].boxes.cls[i])] == 'person':
        x1, y1, x2, y2 = result.boxes.xyxy[i]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,255), 2)
        cv2.putText(img, str(names[int(results[0].boxes.cls[i])]), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
  return img, [x1, y1, x2, y2]
# Show the image with bounding boxes
# cv2_imshow(img)
# cv2.waitKey(0)


In [ ]:
import datetime
def time_now():
  # Get the current UTC time
  current_time = datetime.datetime.utcnow()

  # Calculate the current time's hour, minute, and second components
  current_hour = current_time.hour
  current_minute = current_time.minute
  current_second = current_time.second

  # Calculate the total seconds from the current time to midnight
  total_seconds = (current_hour * 3600) + (current_minute * 60) + current_second

  return total_seconds


In [ ]:
%cd /content/drive/MyDrive/Project_Camera/FireBase/Flask_detection
from flask import Flask, render_template, Response, request
import cv2
import datetime, time
import os, sys
import numpy as np
from threading import Thread
from upload_drive.upload import uploadFile
from pyngrok import ngrok
#Alert to telegram bot
from alert_telegram import send_img_telegram


#Save data in firebase
from firebase_admin import db
import firebase_admin
import time

# cred = firebase_admin.credentials.Certificate('credent.json')
# default_app = firebase_admin.initialize_app(cred, {
# 	'databaseURL':'https://modified-media-390212-default-rtdb.asia-southeast1.firebasedatabase.app/'})
# ref = db.reference("/")


global capture,rec_frame, switch, face, rec, out ,face_frame,frame_to_send
capture=0
face=0
switch=1
rec=0

#make shots directory to save pics
try:
    os.mkdir('./shots')
    os.mkdir('./videos')
except OSError as error:
    pass


#instatiate flask app
app = Flask(__name__, template_folder='./templates')
port_no = 5000
ngrok.set_auth_token("2SvTFgyGsxjL492pDkRxOKLa6Th_28ZxU8zZk9bpGbw9Ti4b2")
public_url =  ngrok.connect(port_no).public_url

# cam_id ='http://192.168.1.5:81/stream'
camera = cv2.VideoCapture('https://40e6-171-224-178-65.ngrok-free.app/video')
def record(out):
    global rec_frame
    while(rec):
        time.sleep(0.05)
        out.write(rec_frame)


last_alert = 0
def detect_face(frame):
    global net, last_alert
    #img, bboxs = detector.findFaces(frame)
    img,bboxs = detection_yolov8_frame(frame)


    return img, bboxs

def alert():
    global last_alert, face
    # New thread to send telegram after 15 seconds
    while(face):
        if ((time_now() - last_alert)) > 150 or last_alert == 0:
              last_alert = time_now()
              try:
                Thread(target = send_img_telegram(frame_to_send), args=[]).start()
              except:
                pass

name = '' # Use to upload image which is this name to google drive

def gen_frames():  # generate frame by frame from camera
    global out, capture,rec_frame,frame_to_send, face_frame, name #use to send image
    while True:
        success, frame = camera.read()
        if success:
            if(face):
                frame,box= detect_face(frame)
                if box:
                    face_frame = True #if has face
                    frame_to_send = frame
                else:
                    face_frame = False
            if(capture):
                capture=0
                now = datetime.datetime.now()
                p = os.path.sep.join(['shots', "shot_{}.png".format(str(now).replace(":",''))])
                cv2.imwrite(p, frame)
                name = "shot_{}.png".format(str(now).replace(":",''))

            if(rec):
                rec_frame=frame
                frame= cv2.putText(frame,"Recording...", (0,25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),4)
                #frame=cv2.flip(frame,1)


            try:
                ret, buffer = cv2.imencode('.jpg', frame)
                frame = buffer.tobytes()
                yield (b'--frame\r\n'
                       b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            except Exception as e:
                pass
        else:
            pass

@app.route('/')
def index():
    return render_template('index.html')


@app.route('/video_feed')
def video_feed():

    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/requests',methods=['POST'])
def tasks():
    global switch,camera
    button_id = request.form['button_id']
    if button_id == "Capture":
        global capture
        capture=1
        print(name)
        Thread(target = uploadFile(name)).start()

    elif button_id == "Face Detection":
        global face, frame_to_send, last_alert
        face=not face
        # if(face):

        #     Thread(target = alert).start()
              #frame_to_send = None
            #thread.start()

    elif  button_id == 'Start/Stop Recording':
            global rec, out
            rec= not rec
            if(rec):
                now=datetime.datetime.now()
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter('videos/vid_{}.avi'.format(str(now).replace(":",'')), fourcc, 20.0, (640, 480))
                #Start new thread for recording the video
                thread = Thread(target = record, args=[out,])
                thread.start()
            elif(rec==False):
                out.release()
    else:
        print('EEll')
        data = request.json.get('data')
        # Process the received data as needed
        print("Received data:", data)
    return "Variable updated successfully"
    # elif request.method=='GET':
    #     return render_template('index.html')

    # return render_template('index.html')
@app.route('/send_data', methods=['POST'])
def receive_data():
    data = request.json.get('data')
    # Process the received data as needed

    print("Received data:", data)
    return "Data received by Flask"
if __name__ == '__main__':
    print(f"To acces the Gloable link please click {public_url}")
    app.run(port=port_no)

camera.release()
cv2.destroyAllWindows()